In [1]:
D.<x> = PolynomialRing(QQ)
R.<x> = PolynomialRing(QQ)
S = PowerSeriesRing(QQ,'x', default_prec=300)
L=LaurentSeriesRing(QQ,'x', default_prec=300)
#S.<x,y>=PowerSeriesRing(QQ,default_prec=300)
# Using "R = PolynomialRing(QQ,'x')" produces an error.
# Yet using S as defined here is fine (see definition of Fraleigh4 below)
# But S = PowerSeriesRing(QQ,'x','y') also produces an error.
R.<x> = PolynomialRing(QQ)
Scplx= PowerSeriesRing(CC,'x',default_prec=300)
def pochhammer(x,n):
    return product ([x+k for k in [0..n-1]])
def cRaleigh2(alpha,beta,nu):
    pchhmrA = pochhammer(alpha,nu)
    pchhmrB = pochhammer(beta,nu)
    fnu=factorial(nu)
    answer = D(pchhmrA*pchhmrB/fnu^2)
    return answer
def term(alpha,beta,p):
    return D(1/(alpha + p) + 1/(beta + p) - 2/(1 + p))
def eRaleigh(alpha, beta, nu):
    return D(sum([term(alpha, beta, p) for p in [0..nu - 1]]))
def FstarRaleigh2(alpha,beta,u,terms):
    fsr = sum([cRaleigh2(alpha, beta, nu)*eRaleigh(alpha,beta,nu)*u^nu for nu in [1..terms]])
    return D(fsr)
def Fraleigh2(alpha,beta,u,terms): 
    return D(sum([cRaleigh2(alpha,beta,nu)*u^nu for nu in [0..terms]]))
def FstarRaleigh3(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fsr2 = FstarRaleigh2(alpha,beta,x,n)
    return D(fsr2)
def Fraleigh4(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fr2 = Fraleigh2(alpha,beta,x,n)
    return D(fr2)
def exNo3c(n,m):
    a1 = S(x)*exp(S(FstarRaleigh3(n,m,x)/Fraleigh4(n,m,x))).O(2*n)
    # S(x) instead of x in df of a1 to avoid an error message
    # after use of the stripQuotationMarks command below.
    a2 = S(a1).O(n+1)
    return a2 # applying S to argument of exp is key.
def J(n,m):
    exn = exNo3c(n+2,m)
    jnm = 1/exn.reverse()
    return jnm.O(n+1) # Jtest is faster than J, and they agree.
def normalJ(n,m):
    f = J(n,m)
    return f.truncate(n+1)
def Jcoefficient(n,m):
    f = J(n+2,m)
    f1 = f.truncate(n+1)
    f1=f1.list()
    f1=Set(f1)
    g1 = f.truncate(n)
    g1 = g1.list()
    g1 = set(g1)
    h = f1.difference(g1)
    h = h.list()
    return h[0]
def xjNewStep(n,m): 
    f=J(n,m)
    fl=f.list()
    return (sum(c*x**i*(m**3*2**6)**i for i,c in enumerate(fl)))
def xjNew(n,m):
    return xjNewStep(n-1,m)
def xjNewCoefficient(n,m):
    if n>0:return (xjNew(n,m)-xjNew(n-1,m))/x^n
    if n==0:return 1
def xjNewTest(n,m):
    return D(xjNewStep(n-1,m))
def j(n,m):return xjNew(n,m)/x
def H4(n,m): #not the published definition (Integers 2021)
    jay=j(n+1,m)
    djay=x*derivative(jay,x) # bc variable is tau, not x; chain rule.
    numerator = djay^2
    denominator = jay*(jay-2^6*m^3)
    return ((S(numerator/denominator)^(1/(m-2))).O(n+1)).polynomial()
def H6(n,m): #different than in draft 4mar21 and Mathematica.
    def base6(n,m):
        jay=j(n+1,m)
        num=(x*derivative(jay,x))^m # chain rule; variable is tau not x.
        den = expand(jay^(m-1)*(j(n+1,m)-2^6*m^3))
        return L((-1)^m*(num/den)).O(n+1)
    return (S(base6(n,m)^(1/(m-2))).polynomial())

def polynomialCoefficient(n,poly):
    return poly.list()[n]
def factorpairToFactor(factorpair):
    (monomial,exponent)=factorpair
    return monomial^exponent
def numericalfactor(polynomial):
    fp=polynomial.factor()
    lst=[factorpairToFactor(factorpair) for factorpair in fp]
    return polynomial/product(lst)
def lastfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.last())
def reduced(polynomial):
    return numericalfactor(polynomial)*lastfactor(polynomial)
def stripQuotationMarks(string):
    var('x')
    return sage_eval(string,locals={'x':x})
def deltaDiamond(n,m):
    return D(S(expand((H4(n+2,m)^3)/j(n+2,m)))).truncate(n+1)
def nieburTau(n):
# The contortions below sidestep mysterious (to me) error messages...
# nieburTau is an algorith for Ramanujan''s tau from the paper
# "A formula for Ramanujan's tau-function" by Douglas Niebur.
# (He spells tau with the Greek letter.)
    def step1(n,k):
        return 35*k^4-52*k^3*n+18*k^2*n^2
    def step2(n,k):
        return sigma(k)*sigma(n-k)
    def step3(n):
        ans=0
        for k in [1..n-1]:
            ans=ans+step1(n,k)*step2(n,k)
        return 24*ans
    return n^4*sigma(n)-step3(n)

print("ok")

ok


In [2]:
for m in [3..12]:
    print("-----------------------------------------------------------------------")
    print(deltaDiamond(5,m))
 

-----------------------------------------------------------------------
4830*x^5 - 1472*x^4 + 252*x^3 - 24*x^2 + x
-----------------------------------------------------------------------
26377322496*x^5 - 71303168*x^4 + 193536*x^3 - 512*x^2 + x
-----------------------------------------------------------------------
736994018750*x^5 - 1041128000*x^4 + 1355100*x^3 - 1480*x^2 + x
-----------------------------------------------------------------------
9341553868800*x^5 - 7616856064*x^4 + 5455872*x^3 - 3072*x^2 + x
-----------------------------------------------------------------------
74765838153790*x^5 - 38002534080*x^4 + 16490460*x^3 - 5432*x^2 + x
-----------------------------------------------------------------------
1312943704637440/3*x^5 - 441718931456/3*x^4 + 41521152*x^3 - 8704*x^2 + x
-----------------------------------------------------------------------
2037064488530526*x^5 - 475635440448*x^4 + 91937916*x^3 - 13032*x^2 + x
--------------------------------------------------------

In [2]:
deltaDiamond(5,3)

4830*x^5 - 1472*x^4 + 252*x^3 - 24*x^2 + x

In [3]:
deltaDiamond(5,4)

26377322496*x^5 - 71303168*x^4 + 193536*x^3 - 512*x^2 + x

In [4]:
deltaDiamond(5,5)

736994018750*x^5 - 1041128000*x^4 + 1355100*x^3 - 1480*x^2 + x

In [6]:
poly=D(deltaDiamond(100,3))
data=[]
ls=poly.list()
for n in [1..len(ls)-1]:
    data=data+[polynomialCoefficient(n,poly)-nieburTau(n)]
print data

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [3]:
import time
import pickle
wfile = open('/Users/barrybrent/10mar21no6.txt','w') # diamond series

data=[]
for m in [3..400]:
    start=time.time()
    print("--------------------------------------------------------------------------------")
    print(m)
    data=data+[[m,deltaDiamond(100,m)]]
    finish=time.time()
    print finish-start
s = pickle.dumps(str(data))
wfile.write(s)
wfile.close()

--------------------------------------------------------------------------------
3
2.87608408928
--------------------------------------------------------------------------------
4
1.94466304779
--------------------------------------------------------------------------------
5
3.6965379715
--------------------------------------------------------------------------------
6
1.89268708229
--------------------------------------------------------------------------------
7
4.2383890152
--------------------------------------------------------------------------------
8
3.20180821419
--------------------------------------------------------------------------------
9
4.53454709053
--------------------------------------------------------------------------------
10
3.02149891853
--------------------------------------------------------------------------------
11
4.56481599808
--------------------------------------------------------------------------------
12
3.94867897034
-----------------------------

5.93999409676
--------------------------------------------------------------------------------
89
6.95016598701
--------------------------------------------------------------------------------
90
5.67790293694
--------------------------------------------------------------------------------
91
7.33038282394
--------------------------------------------------------------------------------
92
5.96950292587
--------------------------------------------------------------------------------
93
7.35056209564
--------------------------------------------------------------------------------
94
4.97282409668
--------------------------------------------------------------------------------
95
7.23117995262
--------------------------------------------------------------------------------
96
5.96640610695
--------------------------------------------------------------------------------
97
6.72828602791
--------------------------------------------------------------------------------
98
5.39387392998
------

6.69485998154
--------------------------------------------------------------------------------
173
7.2993209362
--------------------------------------------------------------------------------
174
6.31372594833
--------------------------------------------------------------------------------
175
8.06392598152
--------------------------------------------------------------------------------
176
6.63309812546
--------------------------------------------------------------------------------
177
7.77432608604
--------------------------------------------------------------------------------
178
5.6369600296
--------------------------------------------------------------------------------
179
7.49518895149
--------------------------------------------------------------------------------
180
7.44633293152
--------------------------------------------------------------------------------
181
7.54967880249
--------------------------------------------------------------------------------
182
6.2677099704

6.47718000412
--------------------------------------------------------------------------------
257
7.87785601616
--------------------------------------------------------------------------------
258
6.62461805344
--------------------------------------------------------------------------------
259
8.20045304298
--------------------------------------------------------------------------------
260
7.72449994087
--------------------------------------------------------------------------------
261
8.46481609344
--------------------------------------------------------------------------------
262
6.36682009697
--------------------------------------------------------------------------------
263
8.06738901138
--------------------------------------------------------------------------------
264
7.56514501572
--------------------------------------------------------------------------------
265
8.26231884956
--------------------------------------------------------------------------------
266
6.60114097

7.2815759182
--------------------------------------------------------------------------------
343
8.85874009132
--------------------------------------------------------------------------------
344
7.3495900631
--------------------------------------------------------------------------------
345
9.02438378334
--------------------------------------------------------------------------------
346
6.52075195312
--------------------------------------------------------------------------------
347
8.49260997772
--------------------------------------------------------------------------------
348
8.1262049675
--------------------------------------------------------------------------------
349
8.51465201378
--------------------------------------------------------------------------------
350
7.39391899109
--------------------------------------------------------------------------------
351
8.92194509506
--------------------------------------------------------------------------------
352
7.51681900024

In [3]:
# interpolating polynomials
import pickle
rfile = open('/Users/barrybrent/10mar21no6.txt','r') # diamond series
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for k in [0..len(s)-1]:
    m = s[k][0]
    poly=D(s[k][1])
    data=data+[[k,poly.degree()]]
print(data) # correctness check: all of the series have the right degree.

[[0, 100], [1, 100], [2, 100], [3, 100], [4, 100], [5, 100], [6, 100], [7, 100], [8, 100], [9, 100], [10, 100], [11, 100], [12, 100], [13, 100], [14, 100], [15, 100], [16, 100], [17, 100], [18, 100], [19, 100], [20, 100], [21, 100], [22, 100], [23, 100], [24, 100], [25, 100], [26, 100], [27, 100], [28, 100], [29, 100], [30, 100], [31, 100], [32, 100], [33, 100], [34, 100], [35, 100], [36, 100], [37, 100], [38, 100], [39, 100], [40, 100], [41, 100], [42, 100], [43, 100], [44, 100], [45, 100], [46, 100], [47, 100], [48, 100], [49, 100], [50, 100], [51, 100], [52, 100], [53, 100], [54, 100], [55, 100], [56, 100], [57, 100], [58, 100], [59, 100], [60, 100], [61, 100], [62, 100], [63, 100], [64, 100], [65, 100], [66, 100], [67, 100], [68, 100], [69, 100], [70, 100], [71, 100], [72, 100], [73, 100], [74, 100], [75, 100], [76, 100], [77, 100], [78, 100], [79, 100], [80, 100], [81, 100], [82, 100], [83, 100], [84, 100], [85, 100], [86, 100], [87, 100], [88, 100], [89, 100], [90, 100], [91, 100

In [3]:
# interpolating polynomials
import pickle
rfile = open('/Users/barrybrent/10mar21no6.txt','r') # diamond series
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
polydata=[]
import time
wfile = open('/Users/barrybrent/20mar21no1.txt','w') # diamond polynomials
for qpower in [0..99]:
    start = time.time()
    data=[]
    for k in [0..len(s)-1]:
        m = s[k][0]
        poly=s[k][1] 
        cf=polynomialCoefficient(qpower,poly)
        data=data+[[m,cf]]
    rl=R.lagrange_polynomial(data)
    polydata=polydata+[[qpower,rl]] 
    finish = time.time()
    print([qpower,finish-start])
t = pickle.dumps(str(polydata))
wfile.write(t)
wfile.close()

[0, 1.109226942062378]
[1, 1.1172280311584473]
[2, 1.1160938739776611]
[3, 1.135072946548462]
[4, 1.058326005935669]
[5, 1.1011030673980713]
[6, 1.1078779697418213]
[7, 1.1228199005126953]
[8, 1.119832992553711]
[9, 1.1297249794006348]
[10, 1.1179141998291016]
[11, 1.1282711029052734]
[12, 1.135056972503662]
[13, 1.087205171585083]
[14, 1.1554689407348633]
[15, 1.1384251117706299]
[16, 1.1606547832489014]
[17, 1.1581590175628662]
[18, 1.1565051078796387]
[19, 1.1529099941253662]
[20, 1.1515650749206543]
[21, 1.1117348670959473]
[22, 1.151766061782837]
[23, 1.1637921333312988]
[24, 1.1606860160827637]
[25, 1.1734890937805176]
[26, 1.174515962600708]
[27, 1.1783421039581299]
[28, 1.1773359775543213]
[29, 1.1866750717163086]
[30, 1.1363890171051025]
[31, 1.1879382133483887]
[32, 1.204625129699707]
[33, 1.2009899616241455]
[34, 1.209561824798584]
[35, 1.217984914779663]
[36, 1.2211389541625977]
[37, 1.2404558658599854]
[38, 1.2394371032714844]
[39, 1.1989059448242188]
[40, 1.26437807083129